# Laboratorio: Búsqueda en Rejilla (Grid Search) con MLflow

## En este laboratorio, tú:
 - Realizarás una búsqueda en rejilla (grid search) usando scikit-learn.
 - Registrarás el mejor modelo en MLflow.
 - Cargarás el modelo guardado para usarlo.

### Preparación del Entorno

Antes de empezar, asegúrate de tener las librerías necesarias. Puedes instalarlas ejecutando la siguiente celda.

Además, para visualizar tus experimentos, abre una terminal, navega a la carpeta de este notebook y ejecuta `mlflow ui`. Esto iniciará la interfaz de MLflow, generalmente en `http://127.0.0.1:5000`.

In [ ]:
# Descomenta y ejecuta si no tienes las librerías instaladas
!pip install mlflow scikit-learn pandas

## Importación de Datos

Cargamos el conjunto de datos de viviendas y creamos una división para entrenamiento y prueba.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import datasets

# Cargar los datos
housing = datasets.fetch_california_housing()
X = pd.DataFrame(housing.data, columns=housing.feature_names)
y = housing.target

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

## Realizar Búsqueda en Rejilla (Grid Search) con scikit-learn

Queremos saber qué combinación de hiperparámetros es la más efectiva. Completa el código a continuación para realizar una <a href="https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV" target="_blank">búsqueda en rejilla usando **`sklearn`**</a>.

Establece **`n_estimators`** a **`[50, 100]`** y **`max_depth`** a **`[3, 5]`**.

In [ ]:
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Diccionario con los nombres de los hiperparámetros y la lista de valores que queremos probar
parameters = {"n_estimators": [50, 100], 
              "max_depth": [3, 5]}

rf = RandomForestRegressor(random_state=42)
grid_rf_model = GridSearchCV(rf, parameters, cv=3)
grid_rf_model.fit(X_train, y_train)

# Obtenemos el mejor estimador (el mejor modelo entrenado)
best_rf = grid_rf_model.best_estimator_

print("Mejores Hiperparámetros Encontrados:")
for p in parameters:
    print(f"Mejor '{p}': {best_rf.get_params()[p]}")

## Registrar el Mejor Modelo con MLflow

Registra el mejor modelo como **`grid-random-forest-model`**, sus parámetros y su métrica MSE bajo una ejecución con el nombre **`RF-Grid-Search`** en un nuevo experimento de MLflow.

In [ ]:
from sklearn.metrics import mean_squared_error

with mlflow.start_run(run_name="RF-Grid-Search") as run:
    # Crea predicciones de X_test usando el mejor modelo
    predictions = best_rf.predict(X_test)

    # Registra el modelo con su nombre
    mlflow.sklearn.log_model(best_rf, "grid-random-forest-model")

    # Registra los mejores parámetros encontrados por GridSearchCV
    mlflow.log_params(grid_rf_model.best_params_)

    # Crea y registra la métrica MSE usando las predicciones de X_test y su valor real y_test
    mse = mean_squared_error(y_test, predictions)
    mlflow.log_metric("mse", mse)
    print(f"Métrica MSE registrada: {mse:.4f}")

    run_id = run.info.run_id
    print(f"Dentro de una ejecución de MLflow con id {run_id}")

## Cargar el Modelo Guardado

Carga el modelo entrenado y ajustado que acabamos de guardar. Verifica que los hiperparámetros de este modelo coincidan con los del mejor modelo que encontramos anteriormente.

In [ ]:
# La URI del modelo tiene el formato "runs:/<run_id>/<artifact_path>"
model_uri = f"runs:/{run_id}/grid-random-forest-model"

# Carga el modelo
loaded_model = mlflow.sklearn.load_model(model_uri)

# Comprueba los parámetros del modelo cargado
print("Parámetros del modelo cargado:")
print(f"- n_estimators: {loaded_model.get_params()['n_estimators']}")
print(f"- max_depth: {loaded_model.get_params()['max_depth']}")

## Opcional: Interactuar con `MlflowClient`

Si el tiempo lo permite, usa `MlflowClient` para interactuar programáticamente con tu ejecución.

In [ ]:
from mlflow.tracking import MlflowClient

# Inicializa el cliente
client = MlflowClient()

# Obtiene los datos de la ejecución que acabamos de realizar
run_data = client.get_run(run_id).data

print("--- Parámetros ---")
print(run_data.params)

print("--- Métricas ---")
print(run_data.metrics)